# E) 2D nonlinear magnetostatics

- **<font color='green'>[RUN & OBSERVE]</font>** $\rightarrow$ the cell should be run directly without modification
- **<font color='orange'>[RUN & PLAY]</font>** $\rightarrow$ the cell can be run directly, but some parameters should be changed interactively
- **<font color='red'>[FILL & RUN]</font>**    $\rightarrow$ the cell should be filled before being run
- **<font color='magenta'>[FILL & PLAY]</font>** $\rightarrow$ the cell should be filled, and then some parameters should be changed interactively.

_______
## 1) Equation

We want to solve the following weak formulation, i.e., find $a\in H^1_0 (\Omega)$ such that

$$ \forall v\in H^1_0(\Omega) , \quad \int_\Omega \nu(|\nabla a|^2) \nabla a \cdot \nabla v = \int_\Omega v j $$

with $\nu$ the magnetic reluctivity, which is
- $ \nu = \nu_0 $ in air and conductors
- $ \nu(b^2) = 100 + 10 \exp(1.8 b^2 ) $ in iron

and $j$ the current density that is
- $ j = 0 $ in air and iron
- $ j= +J $ in positive conductor
- $ j = -J$ in negative conductor
  
with $J = 10\;A/mm^2$. 

________
## 2) Geometry

We consider a closed iron core with a winding. Without airgap, there is a risk of saturation !

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
from utils.myGeometries import gapedInductor
from ngsolve.webgui import Draw
mesh = gapedInductor(airgap = 0, h = 0.01) # no airgap
Draw(mesh.MaterialCF({"condN":1, "iron":2, "condP":3}), mesh)

We can get the lables of the regions...
|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
print(f"Region names : {mesh.GetMaterials()}")  # region names

... and boundaries.

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
print(f"Boundaries names : {mesh.GetBoundaries()}")  # boundaries names

_____
## 3) Nonlinear formulation

### a) Material properties

The magnetic reluctivity in iron depends on the square of the flux density and reads
$$ \nu_{iron}(b^2) = 100 + 10 \exp(1.8 b^2 ) $$

|**<font color='red'>[FILL & RUN]</font>**|
|---|

In [ ]:
from ngsolve import exp, pi

mu0 = 4e-7 * pi
J = 10e6

def nu_iron(b2):
    """ magnetic reluctivity of iron w.r.t b²"""
    return .........................

def dnu_iron_db2(b2):
    """ derivative of the magnetic reluctivity of iron w.r.t b² """
    return .........................

#-----------------------------------------------------------------------
# Plot the BH curve
import numpy as np
import matplotlib.pyplot as plt
b = np.linspace(0,2,100)
plt.plot(nu_iron(b**2) * b, b); plt.xlabel("H (A/m"); plt.ylabel("B (T)")
plt.title("BH curve of iron"); plt.grid(); plt.show()

### b) Residual

We first define the residual (symbolic expression, not a linear or bilinear form)...

|**<font color='red'>[FILL & RUN]</font>**|
|---|

In [ ]:
from ngsolve import grad, dx, CoefficientFunction
R = CoefficientFunction( ((0,1),(-1,0)), dims = (2,2))
curl = lambda u : R * grad(u)

def residual(a, 
             v  # test function
            ):  # returns symbolic expression of the residual 
    ...............................
    return ........................

and its directional derivative  (symbolic expression, not a linear or bilinear form).

|**<font color='red'>[FILL & RUN]</font>**|
|---|

In [ ]:
def d_residual(da,  # trial function
               v,   # test function
               aOld # previous a
              ):    # returns symbolic expression of the residual directional derivative w.r.t aOld + t*da
    ...............................
    return ........................

### c) Simple Newton

Now it's time to apply the Newton method. We have to chose the function space and set up the initial guess.

Then, Newton method consists on three steps:
1. Find $\delta a\in H(\Omega)$ such that $R'(a_{n}, v ; \delta a) = - R(a_{n}, v) \quad \forall v\in H(\Omega)$
2. Update $a_{n+1} = a_{n} + \alpha \delta a $
3. Stop criteria : $ |R(a_{n}, \delta a)| \leq \epsilon_1$,  $ |R(a_{n}, v_{i})| \leq \epsilon_1$, $n\geq n_{max}$,...

|**<font color='red'>[FILL & RUN]</font>**|
|---|

In [ ]:
#-----------------------------------------------------------------------
# initialization
from ngsolve import H1, GridFunction
fes = H1(mesh, order = 1, dirichlet = "out") # finite element space
da, v = fes.TnT()
sol = GridFunction(fes)
resList = []
#-----------------------------------------------------------------------
# loop
from ngsolve import LinearForm, BilinearForm
step = 0.01
tol = 1e-8
maxit = 100
for i in range(maxit):
    # compute and assemble residual and derivatives
    res = LinearForm( .....................).Assemble().vec
    dres = BilinearForm(.....................).Assemble().mat

    # compute norm of the residual
    resList.append(np.linalg.norm(res.FV().NumPy()[fes.FreeDofs()]))
    
    # compute descent direction
    d = ......................

    # update
    sol.vec.data += .............................
    print(f"{i = } | res = {resList[-1]:.5e} | {step = :.2e}")
    
    # simple step control
    if i>0 and resList[-1] < resList[-2]:
        step = min(1, 1.2*step)
    else : 
        step = step/2

    # simple stop criterion based on the norm of residual
    if resList[-1] < tol:
        break

Is it converging? 
- If not how to make it converge?
- If yes, at what rate?
  
|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
plt.semilogy(resList)
plt.grid()
plt.xlabel("Iterations"); plt.ylabel("Norm of the residual");
plt.show()

### d) Post-processing : flux density
We can compute the flux density $B = \text{curl}~a$.
|**<font color='red'>[FILL & RUN]</font>**|
|---|

In [ ]:
B = .................
Draw(B, mesh, 
     vectors={"grid_size" : 50, "offset" : 0.5 },
     settings = { "Objects" : { "Wireframe" : False, "Surface" : False }})

_________
## 4) More elaborated Newton
We can improve by : 
- considering more stopping criteria
- adding proper linesearch
- using NGSolve automatic differentiation capabilities

You can analyze the algorithm in the file `utils\mySolver`, and make some comments.

|**<font color='green'>[RUN & OBSERVE]</font>**|
|---|

In [ ]:
from utils.mySolvers import solveNL

result = solveNL(fes, residual, verbosity = 3)

In [ ]:
plt.semilogy(result["residual"])
plt.grid()
plt.xlabel("Iterations"); plt.ylabel("Norm of the residual");
plt.show()

In [ ]:
Draw(curl(result["solution"]), mesh, 
     vectors={"grid_size" : 50, "offset" : 0.5 },
     settings = { "Objects" : { "Wireframe" : False, "Surface" : False }})